# Web Scrapping estatísticas superliga 2018-2019

In [6]:
# Importando os pacotes necessários
from urllib import request
from bs4 import BeautifulSoup

In [2]:
with request.urlopen('http://cbv-web.dataproject.com/CompetitionMatches.aspx?ID=6') as url:
    page = url.read()

In [3]:
page

b'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://ogp.me/ns/fb#">\r\n<head><meta name="viewport" content="width=device-width, initial-scale=1" /><title>\r\n\tCompetition Site by Data Project\r\n</title><link rel="stylesheet" type="text/css" href="https://fonts.googleapis.com/css?family=Open+Sans:400" /><link href="https://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet" type="text/css" /><link href="https://fonts.googleapis.com/css?family=Open+Sans:300italic,400italic,700italic,300,400,700" rel="stylesheet" type="text/css" /><link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.1/css/font-awesome.min.css" />\r\n    <script type="text/javascript" src="https://code.jquery.com/jquery-3.1.0.min.js"></script>\r\n    <script type="text/javascript" src="https://code.jquery.com/ui/1.12.1/jquery-ui.min.js"></sc

In [25]:
soup = BeautifulSoup(page, 'html.parser')

In [68]:
soup.title

<title>
	Competition Site by Data Project
</title>

### Coletando dados de data e local

In [69]:
rows_data_local = soup.find_all('div', class_="t-col t-col-2 t-col-xs-3 t-col-sm-3 t-hidden-xs t-hidden-sm t-hidden-md Calendar_DIV_Column")

In [70]:
len(rows_data_local)

153

In [71]:
teste = rows_data_local[2]
teste.find_all('span')

[<span id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl4_LB_DataOra">18/01/2019 - 19:30</span>,
 <span id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl4_LB_Palasport">GINASIO MUNICIPAL JOSE LIBERATTI</span>]

In [72]:
texto = teste.find_all('span')

In [73]:
print(texto[0].string+' em '+texto[1].string)

18/01/2019 - 19:30 em GINASIO MUNICIPAL JOSE LIBERATTI


### Coletando ID do jogo

In [161]:
ids = []

for row in rows_data_local:
    result = row.find_all('p', class_='Calendar_p_TextRow Calendar_p_TextRow_Italic')
    ids.append(str(result[0]))

In [163]:
len(ids)

153

### Coletando jogo

In [74]:
rows_times = soup.find_all('div', class_="t-col t-col-2 t-col-sm-3 t-col-md-2 t-hidden-xs Calendar_DIV_Column")

In [87]:
teste1 = rows_times[4]
teste2 = rows_times[5]
texto1 = teste1.find_all('span')
texto2 = teste2.find_all('span')

In [88]:
texto2

[<span id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl4_Label4">SAO CRISTOVAO SAUDE SAO CAETANO</span>]

In [89]:
print(texto1[0].string+' x '+texto2[0].string)

OSASCO VOLEIBOL CLUBE/AUDAX x SAO CRISTOVAO SAUDE SAO CAETANO


### Placar

In [127]:
rows_placar = soup.find_all('div', class_="t-col t-col-1 t-col-xs-4 t-col-sm-3 t-col-md-2 Calendar_DIV_Column")

In [128]:
len(rows_placar)

153

In [129]:
ex = rows_placar[0]
ex

<div class="t-col t-col-1 t-col-xs-4 t-col-sm-3 t-col-md-2 Calendar_DIV_Column">
<p class="Calendar_p_TextRow" onclick="javascript:window.location='MatchStatistics.aspx?mID=532&amp;ID=6&amp;CID=11&amp;PID=9&amp;type=LegList';" style="font-size: 16px;">
<img border="0" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_Img2" src="Images/spacer.gif" width="12"/>
<span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LB_SetCasa"><b>1</b></span>
<span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LBL_Separ"><b> - </b></span>
<span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LB_SetOspiti"><b>3</b></span>
<img border="0" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_Img3" src="Images/Spacer.gif" width="12"/>
</p>
</div>

In [137]:
ex.find_all('span')

[<span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LB_SetCasa"><b>1</b></span>,
 <span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LBL_Separ"><b> - </b></span>,
 <span class="Calendar_Match_Result" id="ctl00_Content_Main_11_userControl_RADLIST_Legs_ctrl0_RADLIST_Matches_ctrl0_LB_SetOspiti"><b>3</b></span>]

### Juntando tudo:

In [101]:
# Coletando todos as datas e locais e armazenando em listas

datas = []
local = []

for row in rows_data_local:
    result = row.find_all('span')
    datas.append(result[0].string)
    local.append(result[1].string)

In [114]:
# Coletando os jogos 
mandante = []
visitante = []

for index in range(len(rows_times)):
    time = rows_times[index].find_all('span')
    if index%2 == 0:
        mandante.append(time[0].string)
    else:
        visitante.append(time[0].string)
    

In [143]:
# Coletando os placares
sets_mandante = []
sets_visitante = []

for row in rows_placar:
    result = row.find_all("span")
    if len(result) == 3:
        sets_mandante.append(result[0].string)
        sets_visitante.append(result[2].string)
    else:
        sets_mandante.append(None)
        sets_visitante.append(None)

### Reunindo todos os dados em um dataframe do Pandas

In [152]:
import pandas as pd

In [164]:
df = pd.DataFrame({
    'id_jogo': ids,
    'data_jogo': datas,
    'local_jogo': local,
    'mandante': mandante,
    'sets_mandante': sets_mandante,
    'sets_visitante': sets_visitante,
    'visitante': visitante,
})